<a href="https://colab.research.google.com/github/Nikhil270703/Nick-/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the training and testing data
train_data = pd.read_csv(train_file_path, sep='\t', names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', names=['label', 'message'])

# Encode labels as binary values
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Separate features and labels
train_messages = train_data['message']
train_labels = train_data['label']
test_messages = test_data['message']
test_labels = test_data['label']


In [ ]:
from tensorflow.keras.layers import TextVectorization

# Define the text vectorization parameters
max_features = 10000  # Max vocabulary size
sequence_length = 150  # Increased sequence length to capture more context

# Create and adapt the vectorizer
vectorizer = TextVectorization(max_tokens=max_features, output_mode='int', output_sequence_length=sequence_length)
vectorizer.adapt(train_messages)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout

# Define the model
model = Sequential([
    Embedding(max_features, 64),  # Increased embedding dimensions
    GlobalAveragePooling1D(),
    Dropout(0.3),  # Adding dropout for regularization
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Vectorize the messages
train_sequences = vectorizer(np.array([[s] for s in train_messages])).numpy()
test_sequences = vectorizer(np.array([[s] for s in test_messages])).numpy()

# Train the model
history = model.fit(train_sequences, train_labels, epochs=20, batch_size=32, validation_data=(test_sequences, test_labels), verbose=1)


In [ ]:
# Function to predict if a message is spam or ham
def predict_message(pred_text):
    # Vectorize the message
    vectorized_text = vectorizer([pred_text])
    # Make the prediction
    prediction = model.predict(vectorized_text)
    # Extract prediction as a scalar and get label
    probability = float(prediction[0][0])
    label = 'spam' if probability > 0.5 else 'ham'
    return [probability, label]

# Test the function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
